### Genetic Algorithm from scratch Example

### Conceptos Fundamentales:

- Cromosoma: En un AG, una solución potencial se representa como un cromosoma, que es una cadena de genes. Cada gen codifica una característica o parámetro de la solución.

- Población: Una población está compuesta por un conjunto de cromosomas. Inicialmente, se crea una población aleatoria de soluciones potenciales.

- Función de Aptitud (Fitness): La función de aptitud evalúa qué tan buena es una solución en términos de la calidad de la solución. Cuanto mayor sea el valor de la función de aptitud, mejor será la solución.

- Selección: En cada generación, los cromosomas se seleccionan para reproducirse en función de su aptitud. Los cromosomas más aptos tienen una mayor probabilidad de ser seleccionados.

- Cruzamiento (Crossover): Los cromosomas seleccionados se combinan para crear nuevos cromosomas (hijos). El cruzamiento mezcla la información genética de los padres.

- Mutación: Ocasionalmente, se realiza una mutación en un cromosoma. La mutación implica cambiar aleatoriamente un gen en un cromosoma para introducir variabilidad en la población.

- Reemplazo: Los hijos generados por el cruzamiento y la mutación reemplazan a algunos de los cromosomas menos aptos en la población, lo que crea la siguiente generación.

- Criterio de Terminación: Se establece un criterio de terminación, como un número máximo de generaciones o una convergencia suficiente, para determinar cuándo detener el algoritmo.

### Implementación General:

- Inicialización: Se crea una población inicial aleatoria de cromosomas que representan soluciones potenciales para el problema.

- Evaluación de Aptitud: Se calcula la aptitud de cada cromosoma en la población utilizando la función de aptitud definida para el problema.

- Selección: Se seleccionan cromosomas para la reproducción en función de su aptitud. Los cromosomas más aptos tienen una mayor probabilidad de ser seleccionados, pero se permite cierta diversidad al seleccionar algunos cromosomas menos aptos.

- Cruzamiento: Los cromosomas seleccionados se cruzan para producir una nueva generación de cromosomas (hijos). El tipo de cruzamiento depende de la implementación y puede variar desde un solo punto de cruzamiento hasta cruzamientos más complejos.

- Mutación: Ocasionalmente, se aplica una mutación a algunos de los cromosomas hijos para introducir variabilidad en la población.

- Reemplazo: Los hijos generados, junto con algunos de los cromosomas padres, reemplazan a los cromosomas menos aptos en la población actual.

- Criterio de Terminación: Se verifica si se ha alcanzado el criterio de terminación. Si no se cumple, se repiten los pasos 3 a 6 para generar la siguiente generación.

- Resultado: La mejor solución encontrada en la última generación se considera la solución aproximada al problema de optimización.

### Esquema Workflow



        Inicio
        |
        |-- 1. Inicialización:
        |    |
        |    |-- Generar una población inicial aleatoria de cromosomas (Vector: Población).
        |    |
        |    |-- Evaluar la aptitud de cada cromosoma en la población (Lista: Aptitudes).
        |
        |-- 2. Criterio de Terminación:
        |    |
        |    |-- ¿Se cumple el criterio de terminación (por ejemplo, número de generaciones o convergencia)? (Parámetro: Criterio)
        |    |
        |    |-- No: Ir a la siguiente etapa (Bucle: Repetición).
        |    |
        |    |-- Sí: Fin del algoritmo. La mejor solución encontrada es el resultado.
        |
        |-- 3. Selección:
        |    |
        |    |-- Seleccionar cromosomas para reproducción basándose en su aptitud (Lista: Aptitudes, Vector: Población).
        |    |
        |    |-- Pueden aplicarse técnicas de selección como ruleta, torneo, o elitismo.
        |
        |-- 4. Cruzamiento (Crossover):
        |    |
        |    |-- Aplicar operadores de cruzamiento para crear nuevos cromosomas (hijos) a partir de los padres seleccionados (Vector: Padres).
        |    |
        |    |-- Los hijos heredan información genética de los padres.
        |
        |-- 5. Mutación:
        |    |
        |    |-- Aplicar mutación en algunos cromosomas hijos con una probabilidad baja (Vector: Hijos).
        |    |
        |    |-- La mutación introduce variabilidad en la población.
        |
        |-- 6. Reemplazo:
        |    |
        |    |-- Reemplazar a algunos de los cromosomas menos aptos en la población actual con los hijos generados (Vector: Población).
        |    |
        |    |-- Los criterios de reemplazo pueden incluir elitismo (mantener los mejores cromosomas) o simplemente reemplazar al azar.
        |
        |-- 7. Evaluación de Aptitud:
        |    |
        |    |-- Calcular la aptitud de cada cromosoma en la nueva población (Lista: Aptitudes, Vector: Población).
        |    |
        |    |-- Regresar al paso 2 (Criterio de Terminación).
        |
        Fin



In [1]:
from random import choices, randint, random, randrange
from typing import List, Callable, Tuple
from collections import namedtuple
from functools import partial

Genome = List[int]
Population = List[Genome]
FitnessFunc = Callable[[Genome], int]
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
Thing = namedtuple('Thing', ['name','value','weight'])

things = [

Thing('a', 500, 2200),
Thing('b', 150, 160),
Thing('c', 60, 350),
Thing('d', 40, 333),
Thing('e', 30, 192),
Thing('f', 500, 2200),

]

more_things = [

Thing('g', 5, 25),
Thing('h', 10, 38),
Thing('i', 15, 80),
Thing('j', 40, 333),
Thing('k', 500, 200),
Thing('l', 100, 70),

] + things


def generate_genome(length: int) -> Genome:
    return choices([0,1], k = length)

def generate_population(size: int, genome_length: int) -> Population:
    return[generate_genome(genome_length) for _ in range(size)]

def fitness(genome: Genome, things: [Thing], weight_limit: int) ->  int:
    if len(genome) != len(things):
        raise ValueError("genome and things must be of the same length")

    weight = 0
    value = 0

    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

        if weight > weight_limit:
            return 0
        
    return value


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2
    )


def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:

    if len(a) != len(b):
        raise ValueError("Genomes must have the same length")

    length = len(a)
    if(length < 2):
        return a,b

    p = randint(1, length -1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:

    for i in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[i] - 1)
    return genome


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc= selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limi: int = 100

) -> Tuple[Population, int]:

    population = populate_func()

    for i in range(generation_limi):
        population = sorted(
            population,
            key=lambda genome: fitness_func(genome),
            reverse = True
        )
    
        if fitness_func(population[0]) >= fitness_limit:
            break
        
        next_generation = population[0:2]

        for j in range(int(len(population)/2)-1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation


    population = sorted(
        population,
        key = lambda genome: fitness_func(genome),
        reverse = True
                                    
    )

    return population, i

populaton, generations = run_evolution(
    populate_func = partial(
        generate_population, size = 500, genome_length = len(more_things)

    ),
    fitness_func = partial(
        fitness, things = more_things, weight_limit = 2000

    ),
    fitness_limit = 750,
    generation_limi = 100
)

def genome_to_things(genome: Genome, things: [Thing]) -> [Thing]:
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing.name]

    return result

print(f"number of generations: {generations}")
print(f"best soluton: {genome_to_things(populaton[0], more_things)}")



number of generations: 0
best soluton: ['g', 'h', 'j', 'k', 'l', 'b', 'c', 'd']
